EXAMEN

Santiago Gutierrez Gonzalez- A00572499


In [4]:
!pip install IPython,display
!pip install agentpy

ERROR: Could not find a version that satisfies the requirement IPython.display (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for IPython.display



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# IMPORTANTE ES NECESARIO TENER EL ARCHIVO streets-2.npy en la misma carpeta si no no corre

In [143]:
import numpy as np
import os


#VERIFICAMOS QUE LAS CALLES SEAN VISIBLES
array = np.load('streets-2.npy')



array = None

try:
    if os.path.exists('streets-2.npy'):
        array = np.load('streets-2.npy')
  
except Exception as e:
    print(f"Error loading file: {e}")

print(array)


[[  1.   1.   1.   1.   5.   5.   1.   1.   4.   4.   4.   4.   5.   1.
    1.   5.   1.   1.   1.   1.   1.   1.   1.   1.   5.   1.   1.   1.
    1.   1.   1.]
 [  1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10. -10.
  -10.   5. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10.
  -10. -10.   1.]
 [  1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10. -10.
  -10.   1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10.
  -10. -10.   1.]
 [  1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10. -10.
  -10.   1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   4. -10. -10.
  -10. -10.   1.]
 [  1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   1. -10. -10. -10.
  -10.   1. -10. -10. -10. -10.   1. -10. -10. -10. -10.   4. -10. -10.
  -10. -10.   1.]
 [  5.   1.   1.   1.   1.   5.   5.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   5.   1.   1.   1.   5.   1.   1.
    1.   1.   1.]
 [  1. -10. -10. -10. -10.  

In [168]:
import agentpy as ap
import numpy as np
import heapq
import matplotlib.pyplot as plt
import IPython
import time

#These two are necessary for sending the data to Unity
import json
import socket
file_loaded= True
city_map =  None

try:
    if os.path.exists('streets-2.npy'):
        city_map = np.load('streets-2.npy')
   
except Exception as e:
    print(f"Error loading file: {e}, the code wont execute because the file is not present")
    file_loaded = False
# initial position of the agent
start = (0, 6)

# This is the data that we are sending through the socket
data = {
    "step": 0,
    "message": "Matrix for floor generation",
    "matrix": city_map.tolist(),
    "AgentPosition": start
}


#######################################
# CLASS FOR AGENT
#######################################
class CityAgent(ap.Agent):
    def setup(self):
        self.actions = [(1, 0), (-1, 0), (0, 1), (0, -1)]  # Actions of the agents to move in the grid
        self.env = self.model.env
        self.path = []  # Path that was created by A*
        self.travel_time = 0  # Track total travel time

    def execute(self):
        if not self.path:
            return

        # Move to the next position in the path
        next_pos = self.path.pop(0)
        self.env.move_to(self, next_pos)
        self.travel_time += abs(self.env.road_conditions[next_pos])  # Accumulate travel time

    def get_position(self):
        return self.env.positions[self]

    def is_valid(self, x, y):
        rows, cols = self.env.road_conditions.shape
        goal_x, goal_y = self.model.p.goal
        return (0 <= x < rows and 0 <= y < cols and self.env.road_conditions[x, y] != -10 and self.env.road_conditions[x,y]!=-1) or (x == goal_x and y == goal_y)

    def AStar(self, start, goal):
        open_list = []
        heapq.heappush(open_list, (0, start))
        g_cost = {start: 0}
        parent = {start: None}

        while open_list:
            _, current = heapq.heappop(open_list)
            if current == goal:
                path = []
                while current:
                    path.append(current)
                    current = parent[current]
                return path[::-1]

            for move in self.actions:
                neighbor = (current[0] + move[0], current[1] + move[1])
                if self.is_valid(*neighbor):
                    new_g = g_cost[current] + self.env.road_conditions[neighbor]
                    if neighbor not in g_cost or new_g < g_cost[neighbor]:
                        g_cost[neighbor] = new_g
                        # here we use the Manhattan distance as heuristic to get f
                        f = new_g + abs(neighbor[0] - goal[0]) + abs(neighbor[1] - goal[1])
                        heapq.heappush(open_list, (f, neighbor))
                        parent[neighbor] = current

        return []  # no path 

#######################################
# CLASS FOR ENVIRONMENT
#######################################
class CityGrid(ap.Grid):
    def setup(self):
        self.road_conditions = self.p.city_map[:, :]

    def move_to(self, agent, new_position):
        self.positions[agent] = new_position

#######################################
# CLASS FOR THE MODEL
#######################################
class CityModel(ap.Model):
    def setup(self):
        self.env = CityGrid(self, shape=self.p.city_map.shape)
        self.agent = CityAgent(self)
        self.env.add_agents([self.agent], positions=[self.p.start])
        self.agent.path = self.agent.AStar(self.p.start, self.p.goal)
        self.agent.path.pop(0)  # Remove start position from path CHECK TO DO
        if self.p.socket:
            self.connect_socket()
        print("Agent Path:", self.agent.path)
    
    def connect_socket(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect(("127.0.0.1", 1102))
        data['AgentPosition'] = self.agent.get_position()
        json_data = json.dumps(data) + "\n" 
        self.sock.send(json_data.encode("utf-8"))
        data["matrix"]= None

    def step(self):
        self.agent.execute()
        data["AgentPosition"] = self.agent.get_position()
        data["step"] += 1
        if self.p.socket:     
            json_string = json.dumps(data)  + "\n" 
            self.sock.send(json_string.encode("utf-8"))

            # We introduce a delay to make the visualization easier to follow
            time.sleep(.75)

    def update(self):
        if self.agent.get_position() == self.p.goal:
            print(f'Agent reached the goal in {self.agent.travel_time} time units, ending simulation.')
            self.stop()
    
    def end(self):
        self.s.close()
        print("Socket closed")

#######################################
# VISUALIZATION FUNCTION
#######################################
def animation_plot(model, ax):
    grid = model.p.city_map.copy()
    agent_pos = model.env.positions[model.agent]
    grid[agent_pos] = 99  # Mark agen position
    goal_pos = model.p.goal
    grid[goal_pos]= 100 # Mark goal 

    # Color map
    color_dict = {
        1: 'white',       # Good road
        2: '#8B4513',     # Dirt road
        4: 'orange',      # Cracked road
        5: 'red',         # Potholes
        -1: 'black',      # Road closures
        -10: 'blue',      # Buildings
        99: 'green',       # Agent
        100: 'magenta'      # Goal
    }

    # this is for assigning the colors
    cmap = plt.cm.colors.ListedColormap([color_dict[key] for key in sorted(color_dict.keys())])
    bounds = list(sorted(color_dict.keys())) + [max(color_dict.keys()) + 1]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

    ax.imshow(grid, cmap=cmap, norm=norm)
    ax.set_title(f"City Streets Simulation\nTravel Time: {model.agent.travel_time} units \nAgent Position: {model.agent.get_position()}")
    
    # Legend
    legend_patches = [plt.matplotlib.patches.Patch(color=color_dict[key], label=f"{key}: {desc}") 
                      for key, desc in {
        1: "Good Road",
        2: "Dirt Road",
        4: "Cracked Road",
        5: "Potholes",
        -1: "Road Closures",
        -10: "Buildings",
        99: "Agent",
        100: "Goal"
    }.items()]

    ax.legend(handles=legend_patches, loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0., title="Legend", ncol=1)
    plt.tight_layout(pad=2)


#######################################
# EXECUTION AND VISUALIZATION
#######################################


# Test different path
# city_map[5, 30] = 2  # Start
# city_map[5,5] = -1
# city_map[15, 15] = 1

parameters = {
    'city_map': city_map,
    'start': start,
    'goal': (21, 18),
    'socket':False
}

fig, ax = plt.subplots(figsize=(10, 8))
model = CityModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())


Agent Path: [(0, 7), (0, 8), (0, 9), (0, 10), (1, 10), (2, 10), (3, 10), (4, 10), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (5, 20), (5, 21), (5, 22), (5, 23), (5, 24), (5, 25), (5, 26), (5, 27), (5, 28), (5, 29), (5, 30), (6, 30), (7, 30), (8, 30), (9, 30), (10, 30), (11, 30), (12, 30), (13, 30), (14, 30), (15, 30), (16, 30), (17, 30), (18, 30), (19, 30), (20, 30), (20, 29), (20, 28), (20, 27), (20, 26), (20, 25), (20, 24), (20, 23), (20, 22), (20, 21), (20, 20), (20, 19), (20, 18), (21, 18)]
Agent reached the goal in 87.0 time units, ending simulation.


<Figure size 640x480 with 0 Axes>

### Propose Appropriate Tools and Techniques to Solve the Problem
   For the solution of this problem I decided to implement A*, the reason for it is that is very straighforward to implement and guarantees a solution in a good time at the same time that the heuristic is easily obtained throughout using the manhattan distance from the current matrix,This is specially true since we know in advance all of the data regarding the city and this won't change dinamically which guarantees us finding an optimal path in a single run,The possible different paths such as roads, potholes can be used as numeric costs to find the guaranteed least-cost path that we can find,Comparing it with other options that we saw throught the course such as Q-Learning and A Weighted there are certain advantages,With A* we do not have to worry about a reinforcement learning period where will explore and try to find the goal, which takes away the work of finding the optimal number of training episodes, we find the optimal solution first try,In the other hand A weighted could probably be faster, but in this case computational cost is no issue considering the small graph that we have, and this weighted could mean a sacrifice in finding the actual optimal solution.

### Clearly Describe the Agent and Its Environment

#### Agent
  State:
    Position is a (row, column) tuple that identifies the agents location on the grid.
    Time traveled is used to measure the total cost so far.

  Actions:
    Move up, down, left, or right,From (r, c), up is (r-1, c), down is (r+1, c), left is (r, c-1), right is (r, c+1).

  Goal:
    Reach the work location (goal cell) from the home location (start cell) while minimizing the total travel cost.

  Utility:
    Defined as the negative of the sum of path costs,Maximizing negative cost is equivalent to minimizing cost.

#### Environment
  State:
    Each cell is either passable (with a positive cost: 1, 2, 4, 5) or impassable (marked with negative values like -1, -10).

  Actions:
    The city is static; it does not change or take actions.

  Goal and Utility:
    The environment has no goal or utility, it only provides costs and obstacles.

### Implementation of the Agent and Visualization of the Environment
  *On the code*

### Integrate Chosen Tools or Techniques to Select the Optimal Path
  *On the code*

### Provide the Best Route Found With Your Implementation
  The time taken should be 77 units before entering the garage, plus an additional cost of 10 for entering the garage, totaling 87.

### Show an Animation of the Agent’s Movement
  *On the code*
